# Model configs and params examples

There are some common methods for RecTools models that simplify framework integration with experiment trackers (e.g. MlFlow) and allow running experiments from configs.
They include:

* `from_config`
* `get_config`
* `get_params`

We also allow saving and loading models with methods:

* `save`
* `load`

In this example we will show basic usage for all of these methods as well as config examples for our models.

In [6]:
from datetime import timedelta

from rectools.models import (
    ImplicitItemKNNWrapperModel, 
    ImplicitALSWrapperModel, 
    EASEModel, 
    PopularInCategoryModel, 
    PopularModel, 
    RandomModel, 
    LightFMWrapperModel,
    PureSVDModel,
)

## Basic usage

`from_config` methods allows model initialization from a dictionary of model hyper-params.

In [12]:
config = {
    "popularity": "n_interactions",
    "period": timedelta(weeks=2),
}
model = PopularModel.from_config(config)

`get_config` method returns a dictionary of model hyper-params. In contrast to the previous method, here you will get a full list of model parameters, even the ones that were not specified during model initialization but instead were set to their default values.

In [53]:
model.get_config()

{'verbose': 0,
 'popularity': <Popularity.N_INTERACTIONS: 'n_interactions'>,
 'period': {'days': 14},
 'begin_from': None,
 'add_cold': False,
 'inverse': False}

You can directly use output of `get_config` method to create new model instances using `from_config` method. New instances will have exactly the same hyper-params as the source model.

In [6]:
source_config = model.get_config()
new_model = PopularModel.from_config(source_config)

To get model config in json-compatible format pass `simple_types=True`. See how `popularity` parameter changes for the Popular model in the example below:

In [57]:
model.get_config(simple_types=True)

{'verbose': 0,
 'popularity': 'n_interactions',
 'period': {'days': 14},
 'begin_from': None,
 'add_cold': False,
 'inverse': False}

`get_params` method allows to get model hyper-parameters as a flat dictionary which is often more convenient for experiment trackers. 


Don't forget to pass `simple_types=True` to make the format json-compatible. Note that you can't initialize a new model from the output of this method.

In [56]:
model.get_params(simple_types=True)

{'verbose': 0,
 'popularity': 'n_interactions',
 'period.days': 14,
 'begin_from': None,
 'add_cold': False,
 'inverse': False}

`save` and `load` model methods do exactly what you would expect from their naming :)
Fit model to dataset before saving. Weights will be loaded during `load` method.

In [13]:
model.save("pop_model.pkl")

220

In [14]:
loaded = PopularModel.load("pop_model.pkl")

## Configs examples for all models

### ItemKNN
`ImplicitItemKNNWrapperModel` is a wrapper.   
Use "model" key in config to specify wrapped model class and params:

Specify which model you want to wrap under the "model.cls" key. Options are:
- "TFIDFRecommender"
- "CosineRecommender"
- "BM25Recommender"
- "ItemItemRecommender"
- A path to a class (including any custom class) that can be imported. Like "implicit.nearest_neighbours.TFIDFRecommender"

Specify wrapped model hyper-params under the "model.params" key

In [58]:
model = ImplicitItemKNNWrapperModel.from_config({
    "model": {
        "cls": "TFIDFRecommender",  # or "implicit.nearest_neighbours.TFIDFRecommender"
        "params": {"K": 50, "num_threads": 1}
    }
})

In [59]:
model.get_params(simple_types=True)

{'verbose': 0,
 'model.cls': 'TFIDFRecommender',
 'model.params.K': 50,
 'model.params.num_threads': 1}

### iALS
`ImplicitALSWrapperModel` is a wrapper.  
Use "model" key in config to specify wrapped model class and params:  

Specify which model you want to wrap under the "model.cls" key. Since there is only one default model, you can skip this step. "implicit.als.AlternatingLeastSquares" will be used by default. Also you can pass a path to a class (including any custom class) that can be imported.

Specify wrapped model hyper-params under the "model.params" key.  

Specify wrapper hyper-params under relevant keys.

In [8]:
config = {
    "model": {
        # "cls": "AlternatingLeastSquares",  # will work too
        # "cls": "implicit.als.AlternatingLeastSquares",  # will work too
        "params": {
            "factors": 16,
            "num_threads": 2,
            "iterations": 2,
            "random_state": 32
        },
    },
    "fit_features_together": True,
}
model = ImplicitALSWrapperModel.from_config(config)

In [9]:
model.get_params(simple_types=True)

{'verbose': 0,
 'model.cls': 'AlternatingLeastSquares',
 'model.params.factors': 16,
 'model.params.regularization': 0.01,
 'model.params.alpha': 1.0,
 'model.params.dtype': 'float32',
 'model.params.use_native': True,
 'model.params.use_cg': True,
 'model.params.use_gpu': False,
 'model.params.iterations': 2,
 'model.params.calculate_training_loss': False,
 'model.params.num_threads': 2,
 'model.params.random_state': 32,
 'fit_features_together': True}

### EASE

In [65]:
config = {
    "regularization": 100,
    "verbose": 1,
}
model = EASEModel.from_config(config)

In [66]:
model.get_params(simple_types=True)

{'verbose': 1, 'regularization': 100.0, 'num_threads': 1}

### PureSVD

In [67]:
config = {
    "factors": 32,
}
model = PureSVDModel.from_config(config)

In [68]:
model.get_params(simple_types=True)

{'verbose': 0,
 'factors': 32,
 'tol': 0.0,
 'maxiter': None,
 'random_state': None}

### LightFM

`LightFMWrapperModel` is a wrapper.  
Use "model" key in config to specify wrapped model class and params:  

Specify which model you want to wrap under the "model.cls" key. Since there is only one default model, you can skip this step. "LightFM" will be used by default. Also you can pass a path to a class (including any custom class) that can be imported. Like "lightfm.lightfm.LightFM"

Specify wrapped model hyper-params under the "model.params" key.  

Specify wrapper hyper-params under relevant keys.

In [74]:
config = {
    "model": {
        # "cls": "lightfm.lightfm.LightFM",  # will work too 
        # "cls": "LightFM",  # will work too 
        "params": {
            "no_components": 16,
            "learning_rate": 0.03,
            "random_state": 32,
            "loss": "warp"
        },
    },
    "epochs": 2,
}
model = LightFMWrapperModel.from_config(config)

In [75]:
model.get_params(simple_types=True)

{'verbose': 0,
 'model.cls': 'LightFM',
 'model.params.no_components': 16,
 'model.params.k': 5,
 'model.params.n': 10,
 'model.params.learning_schedule': 'adagrad',
 'model.params.loss': 'warp',
 'model.params.learning_rate': 0.03,
 'model.params.rho': 0.95,
 'model.params.epsilon': 1e-06,
 'model.params.item_alpha': 0.0,
 'model.params.user_alpha': 0.0,
 'model.params.max_sampled': 10,
 'model.params.random_state': 32,
 'epochs': 2,
 'num_threads': 1}

### Popular

In [76]:
from datetime import timedelta
config = {
    "popularity": "n_interactions",
    "period": timedelta(weeks=2),
}
model = PopularModel.from_config(config)

In [77]:
model.get_params(simple_types=True)

{'verbose': 0,
 'popularity': 'n_interactions',
 'period.days': 14,
 'begin_from': None,
 'add_cold': False,
 'inverse': False}

### Popular in category

In [78]:
config = {
    "popularity": "n_interactions",
    "period": timedelta(days=1),
    "category_feature": "genres",
    "mixing_strategy": "group"
}
model = PopularInCategoryModel.from_config(config)

In [79]:
model.get_params(simple_types=True)


{'verbose': 0,
 'popularity': 'n_interactions',
 'period.days': 1,
 'begin_from': None,
 'add_cold': False,
 'inverse': False,
 'category_feature': 'genres',
 'n_categories': None,
 'mixing_strategy': 'group',
 'ratio_strategy': 'proportional'}

### Radom

In [80]:
config = {
    "random_state": 32,
}
model = RandomModel.from_config(config)

In [81]:
model.get_params(simple_types=True)

{'verbose': 0, 'random_state': 32}